In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis

def sma(window, columns): # Signal Magnitude Area(주로 움직임의 크기를 나타냄)
    return np.sum(np.abs(window[columns]), axis=1).mean()

def energy(window, column): # 에너지 계산(신호의 강도를 나타냄)
    return np.sum(window[column]**2) / len(window[column])

def iqr(window, column): # 데이터의 75%와 25% 사이의 범위를 계산하여 데이터의 분산 정도를 나타냄
    return np.percentile(window[column], 75) - np.percentile(window[column], 25)

def mad(window, column): # Median Absolute Deviation(데이터 값이 중앙값에서 얼마나 벗어나는지 나타냄)
    return np.median(np.abs(window[column] - np.median(window[column])))

def meanFreq(window, column):  # 푸리에 변환 계산 후 주파수 가중평균 반환(주파수 도메인에서 데이터의 주요 변화를 나타냄)
    # 주어진 데이터에 푸리에 변환을 적용한 뒤, 주파수와 진폭의 가중평균을 계산
    fft_vals = np.fft.fft(window[column])
    freqs = np.fft.fftfreq(len(fft_vals))
    mag = np.abs(fft_vals)
    return np.sum(freqs * mag) / np.sum(mag)

def bandsEnergy(window, column, num_bins=64): # 주파수 대역 에너지 계산(푸리에 변환 후, 상위 num_bins 내의 에너지를 계산)
    fft_vals = np.fft.fft(window[column])
    mag = np.abs(fft_vals)[:num_bins]
    return np.sum(mag**2) / num_bins

def angle(vec1, vec2): # 두 벡터 사이의 각도 계산
    dot_product = np.dot(vec1, vec2)
    magnitude = np.linalg.norm(vec1) * np.linalg.norm(vec2)
    return np.arccos(dot_product / magnitude) if magnitude != 0 else 0

def process_motion(file_path, window_size=100, stride=50): # 윈도우 크기: 100, 윈도우 이동 간격: 50
    data = pd.read_csv(file_path)
    data.columns = data.columns.str.strip() # 열 이름의 공백 제거

    windows = []
    for i in range(0, len(data) - window_size, stride): # stride만큼 건너뛰면서 윈도우를 만듬
        window = data.iloc[i:i + window_size] # 데이터에서 window_size 만큼의 데이터를 슬라이딩 방식으로 잘라 윈도우 생성.

        features = {}

        for sensor in ['Acceleration', 'Gyro']:
            for axis in ['X', 'Y', 'Z']: # 가속도(Acceleration)와 자이로(Gyro)의 X, Y, Z 축 데이터를 기준으로 다양한 특징(mean, std 등) 계산
                col = f"{sensor}{axis}"
                features[f'mean_{sensor.lower()}_{axis.lower()}'] = window[col].mean()
                features[f'std_{sensor.lower()}_{axis.lower()}'] = window[col].std()
                features[f'mad_{sensor.lower()}_{axis.lower()}'] = mad(window, col)
                features[f'max_{sensor.lower()}_{axis.lower()}'] = window[col].max()
                features[f'min_{sensor.lower()}_{axis.lower()}'] = window[col].min()
                features[f'energy_{sensor.lower()}_{axis.lower()}'] = energy(window, col)
                features[f'bandsEnergy_{sensor.lower()}_{axis.lower()}'] = bandsEnergy(window, col)
                features[f'iqr_{sensor.lower()}_{axis.lower()}'] = iqr(window, col)
                features[f'meanFreq_{sensor.lower()}_{axis.lower()}'] = meanFreq(window, col)
                features[f'skewness_{sensor.lower()}_{axis.lower()}'] = skew(window[col])
                features[f'kurtosis_{sensor.lower()}_{axis.lower()}'] = kurtosis(window[col])

        features['sma_acc'] = sma(window, ['AccelerationX', 'AccelerationY', 'AccelerationZ']) # 현재 구간(window) 동안의 전체 가속도 크기
        features['sma_gyro'] = sma(window, ['GyroX', 'GyroY', 'GyroZ']) # 현재 구간(window) 동안의 전체 회전 운동 크기

        acc_vector = [window['AccelerationX'].mean(), window['AccelerationY'].mean(), window['AccelerationZ'].mean()] # 가속도 데이터의 평균으로 구성된 벡터
        gyro_vector = [window['GyroX'].mean(), window['GyroY'].mean(), window['GyroZ'].mean()] # 자이로스코프 데이터의 평균으로 구성된 벡터
        features['angle_acc_gyro'] = angle(acc_vector, gyro_vector) # 가속도와 자이로스코프 벡터 간의 각도

        features['label'] = 'shaking_lr' # 각 데이터셋에 해당하는 label 입력

        windows.append(features)

    return pd.DataFrame(windows)

# 6개의 dataset을 사용. 아래의 6개의 data는 모두 같은 행동의 data
# 각각의 dataset은 15초 동안 특정 행동을 수행하여 추출한 data.
file_paths = [
    'sensordata1.csv',
    'sensordata2.csv',
    'sensordata3.csv',
    'sensordata4.csv',
    'sensordata5.csv',
    'sensordata6.csv'
]

window_size = 100  # 윈도우 크기: 100
stride = 50  # 윈도우 이동 간격: 50

all_motion_features = []

for file_path in file_paths:
    motion_features = process_motion(file_path, window_size, stride)
    all_motion_features.append(motion_features)

motion_features_df = pd.concat(all_motion_features, ignore_index=True)  # 각 파일별로 처리한 결과를 하나의 데이터프레임으로 결합.

output_file_path = "shaking_lr_dataset.csv"
motion_features_df.to_csv(output_file_path, index=False)

output_file_path

'shaking_lr_dataset.csv'